### 1 复现课堂代码

In [1]:
import numpy as np
import pandas as pd
import jieba
import random
from functools import partial

In [2]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [3]:
simple_grammar

'\nsentence => noun_phrase verb_phrase\nnoun_phrase => Article Adj* noun\nAdj* => null | Adj Adj*\nverb_phrase => verb noun_phrase\nArticle =>  一个 | 这个\nnoun =>   女人 |  篮球 | 桌子 | 小猫\nverb => 看着   |  坐在 |  听着 | 看见\nAdj =>  蓝色的 | 好看的 | 小小的\n'

In [4]:
def adj(): return np.random.choice(list(map(lambda s: s.strip(), '蓝色的 | 好看的 | 小小的'.split('|'))))

In [5]:
adj()

'小小的'

In [6]:
def adj_star(): return np.random.choice([lambda : '', lambda : adj()+adj_star()])()

In [7]:
adj_star()

''

In [8]:
list(map(lambda s: s.strip(), '蓝色的 | 好看的 | 小小的'.split('|')))

['蓝色的', '好看的', '小小的']

In [9]:
adj_grammar = """
Adj* => null | Adj Adj*
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [10]:
def create_grammar(grammar_str, split_='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        k, v = line.split(split_)
        grammar[k.strip()] = [s.split() for s in v.split('|')]
    return grammar

In [11]:
create_grammar(adj_grammar)

{'Adj*': [['null'], ['Adj', 'Adj*']], 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [12]:
def generate(gram, target):
    if target not in gram: return target
    expanded = [generate(gram, t) for t in random.choice(gram[target])]
    return ''.join(s for s in expanded if s != 'null')

In [13]:
example_grammar = create_grammar(simple_grammar)
example_grammar

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [14]:
generate(example_grammar, 'sentence')

'一个篮球坐在这个小猫'

In [15]:
from collections import Counter

In [16]:
with open('../../Lesson01/article_9k.txt', 'r') as f:
    articles = list(map(lambda s: s.strip('\n'), f.readlines()))

In [17]:
articles[:2]

['此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息',
 '骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台']

In [18]:
with_jieba_cut = Counter(jieba.cut(articles[1]))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\xxx\AppData\Local\Temp\jieba.cache
Loading model cost 0.794 seconds.
Prefix dict has been built succesfully.


In [19]:
with_jieba_cut

Counter({'骁龙': 4,
         '835': 2,
         '作为': 2,
         '唯一': 1,
         '通过': 1,
         'Windows10': 3,
         '桌面': 3,
         '平台': 3,
         '认证': 1,
         '的': 8,
         'ARM': 1,
         '处理器': 1,
         '高通': 2,
         '强调': 1,
         '不会': 1,
         '因为': 1,
         '只': 1,
         '考虑': 1,
         '性能': 2,
         '而': 1,
         '去': 1,
         '屏蔽掉': 1,
         '小': 1,
         '核心': 1,
         '相反': 1,
         '他们': 1,
         '正': 1,
         '联手': 1,
         '微软': 2,
         '找到': 1,
         '一种': 1,
         '适合': 1,
         '兼顾': 1,
         '和': 1,
         '功耗': 1,
         '完美': 1,
         '方案': 2,
         '报道': 1,
         '称': 1,
         '已经': 1,
         '拿到': 1,
         '了': 2,
         '一些': 1,
         '新': 1,
         '源码': 1,
         '以便': 1,
         '更好': 1,
         '地': 1,
         '理解': 1,
         'biglittle': 1,
         '架构': 1,
         '资料': 1,
         '显示': 1,
         '一款': 1,
         '集成': 1,
   

In [20]:
articles[1]

'骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台'

In [21]:
def cut(string): return list(jieba.cut(string))

In [22]:
token = []
for sentence in articles[:10000]:
    token += cut(sentence)

In [23]:
words_count = Counter(token)

In [24]:
words_count.most_common(100)

[('的', 184244),
 ('在', 47370),
 ('了', 36722),
 ('和', 30809),
 ('是', 30283),
 ('月', 18711),
 ('也', 15995),
 ('年', 15971),
 ('有', 14714),
 ('为', 14448),
 ('等', 14340),
 ('将', 14060),
 ('对', 13074),
 ('与', 12568),
 ('日', 12322),
 ('中', 11117),
 ('中国', 11036),
 ('6', 10477),
 ('上', 10192),
 ('不', 10027),
 ('他', 9530),
 ('都', 9447),
 ('发展', 8795),
 ('企业', 8584),
 ('就', 8537),
 ('到', 8338),
 ('市场', 8095),
 ('但', 7729),
 ('这', 7658),
 ('被', 7575),
 ('从', 7513),
 ('并', 7412),
 ('人', 7339),
 ('后', 7084),
 ('公司', 6915),
 ('一个', 6772),
 ('说', 6703),
 ('新', 6467),
 ('表示', 6309),
 ('要', 6276),
 ('还', 6245),
 ('会', 6179),
 ('个', 6176),
 ('我', 6141),
 ('而', 6090),
 ('进行', 5802),
 ('我们', 5742),
 ('记者', 5734),
 ('以', 5615),
 ('5', 5569),
 ('工作', 5135),
 ('没有', 5000),
 ('美国', 4840),
 ('下', 4741),
 ('更', 4739),
 ('通过', 4720),
 ('大', 4704),
 ('让', 4701),
 ('可以', 4681),
 ('经济', 4670),
 ('时', 4654),
 ('目前', 4645),
 ('国家', 4628),
 ('项目', 4538),
 ('问题', 4422),
 ('创新', 4416),
 ('多', 4410),
 ('已经', 4391),
 ('建设

In [29]:
def prob_1(word, gram_1):
    return (gram_1.get(word, 0) + 1) / (sum(gram_1.values()) + len(gram_1.keys()))

In [30]:
prob_1('我们', words_count)

0.0014985669285092508

In [31]:
token[:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']

In [32]:
token_2_gram = [token[i]+token[i+1] for i in range(len(token)-1)]

In [33]:
token_2_gram[:10]

['此外自', '自本周', '本周6', '6月', '月12', '12日起', '日起除', '除小米', '小米手机', '手机6']

In [34]:
words_count_2 = Counter(token_2_gram)
words_count_2.most_common(10)

[('6月', 8502),
 ('2016年', 2773),
 ('2017年', 2496),
 ('的是', 2250),
 ('5月', 2111),
 ('也是', 2054),
 ('nannan', 1740),
 ('都是', 1590),
 ('自己的', 1496),
 ('更多', 1490)]

In [36]:
def prob_2(word1, word2, gram_2):
    return (gram_2.get(word1+word2, 0) + 1) / (sum(gram_2.values()) + len(gram_2.keys()))

In [37]:
prob_2('在', '吃饭', words_count_2)

3.753629055491212e-07

In [38]:
prob_2('我们', '在', words_count_2)

3.265657278277354e-05

In [40]:
def get_probability(sentence, gram1=words_count, gram2=words_count_2):
    words = cut(sentence)
    
    p = prob_1(words[0], words_count)
    for i in range(len(words)-1):
        p *= (prob_2(words[i], words[i+1], gram2) / prob_1(words[i], gram1))
    return p

In [41]:
get_probability('小明今天抽奖抽到一台苹果手机')

1.7872759224733878e-16

In [42]:
get_probability('小明今天抽奖抽到一架波音飞机')

1.3488487755820422e-16

In [43]:
get_probability('洋葱奶昔来一杯')

1.532407417264307e-11

In [44]:
for sen in [generate(gram=example_grammar, target='sentence') for _ in range(10)]:
    print('sentence: {} with prob: {}'.format(sen, get_probability(sen)))

sentence: 一个好看的蓝色的小猫听着这个好看的蓝色的小小的桌子 with prob: 1.0767384485605308e-40
sentence: 这个小小的小猫看见一个女人 with prob: 5.427483883781073e-15
sentence: 这个女人看见这个篮球 with prob: 1.1975531604022357e-14
sentence: 这个女人看着一个蓝色的好看的蓝色的桌子 with prob: 1.0573577719098792e-32
sentence: 一个好看的蓝色的小小的女人看见一个桌子 with prob: 5.341999177522378e-28
sentence: 一个桌子看见一个蓝色的桌子 with prob: 2.5516548817193607e-20
sentence: 这个篮球看着一个好看的女人 with prob: 8.763846955114184e-21
sentence: 这个女人看着一个好看的小小的好看的女人 with prob: 1.0872586852539898e-27
sentence: 这个桌子坐在一个小小的蓝色的篮球 with prob: 1.0085857305594266e-21
sentence: 一个好看的蓝色的蓝色的小猫坐在一个小猫 with prob: 1.1676278040506785e-31


In [45]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]
for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = map(get_probability, [s1, s2])
    
    better = s1 if p1 > p2 else s2
    print('{} is more possible'.format(better))
    print('-'*4+'{} with probability {}'.format(s1, p1))
    print('-'*4+'{} with probability {}'.format(s2, p2))

明天晚上请你吃大餐，我们一起吃苹果 is more possible
----今天晚上请你吃大餐，我们一起吃日料 with probability 7.122997470034404e-27
----明天晚上请你吃大餐，我们一起吃苹果 with probability 7.122997470034404e-27
真是一只好看的小猫 is more possible
----真事一只好看的小猫 with probability 1.3294315541827732e-18
----真是一只好看的小猫 with probability 9.278668757725317e-16
今晚我去吃火锅 is more possible
----今晚我去吃火锅 with probability 2.8217027243221217e-13
----今晚火锅去吃我 with probability 3.75410871749077e-14
养乐多绿来一杯 is more possible
----洋葱奶昔来一杯 with probability 1.532407417264307e-11
----养乐多绿来一杯 with probability 1.349911773855535e-07


### 2 问答和编程练习

#### 2.1 基础理论部分

0.Can you come up out 3 sceneraies which use AI methods? <br>
Ans: 语音助手、无人驾驶、机器翻译

1.How do we use Github; Why do we use Jupyter and Pycharm? <br>
Ans: 
   * use Github: 基本步骤：git add ... -> git commit -m ... -> git push origin ...
   * 使用 Jupyter 可以快速方便地实现初步想法进行实验、方便进行可视化
   * 使用 Pycharm 进行项目真正的开发，稳定而有效率

2.What's the Probability Model? <br>
Ans: 概率模型是用来描述不同随机变量之间关系的数学模型，通常情况下刻画了一个或多个随机变量之间的相互非确定性的概率关系。从数学上讲，该模型通常被表达为 $\displaystyle (Y,P)$，其中 $\displaystyle Y$是观测集合用来描述可能的观测结果， $\displaystyle P$是$\displaystyle Y$对应的概率分布函数集合。若使用概率模型，一般而言需假设存在一个确定的分布$\displaystyle P$生成观测数据$\displaystyle Y$ (摘自 https://zh.wikipedia.org/wiki/%E6%A6%82%E7%8E%87%E6%A8%A1%E5%9E%8B)

3.Can you came up with some sceneraies at which we could use Probability Model? <br>
Ans: 使用隐马尔可夫模型进行词性标注、使用贝叶斯模型进行分类

4.Why do we use probability and what's the difficult points for programming based on parsing and pattern match? <br>
Ans: 使用概率会得到通用性更强的语言模型；基于解析和模式匹配的模型难以适应各种各样的模式，针对每一种模式都要重新写一套代码，比较繁琐、冗余

5.What's the Language Model? <br>
Ans: 简单来说语言模型就是计算一个句子的概率的模型

6.Can you came up with some sceneraies at which we could use Language Model? <br>
Ans: 机器翻译、语音识别

7.What's the 1-gram language model? <br>
Ans: 假设我们有一个 $m$ 个词组成的句子，我们希望计算得到这个句子的概率 $p(w_1, w_2, ..., w_m)$，1-gram 语言模型假设当前词的出现仅和自己相关，根据链式法则，有：
$$p(w_1,w_2,...,w_m)=p(w_1)*p(w_2)*p(w_3)......p(w_m)$$

8.What's the disadvantages and advantages of 1-gram language model? <br>
Ans: 1-gram 语言模型的优点是计算简单开销小；缺点是由于假设失去了句子中词与词之间的互相关联信息，丢失了大量有用信息，也就导致了模型产生了偏差

9.What't the 2-gram models? <br>
Ans: 假设我们有一个 $m$ 个词组成的句子，我们希望计算得到这个句子的概率 $p(w_1, w_2, ..., w_m)$，2-gram 语言模型假设当前词的出现仅和前一个词相关，根据链式法则，有：
$$p(w_1,w_2,...,w_m)=p(w_1)*p(w_2|w_1)*p(w_3|w_2)......p(w_m|w_{m-1})$$

#### 2.2 编程实践部分

##### 2.2.1 设计自己的句子生成器

In [46]:
def generate_n(grammar, target, n):
    res = []
    for _ in range(n):
        res.append(generate(grammar, target))
    return res

In [47]:
singer = """
singer = 姓名 程度副词* 擅长 特点
姓名 = 华晨宇 | 毛不易
程度副词* = null | 程度副词 程度副词*
程度副词 = 很 | 非常 | 特别 | 极其
擅长 = 喜欢 | 善于 | 擅长
特点 = 作词 | 作曲 | 喝酒 | 怼人
"""

In [48]:
singer_grammar = create_grammar(singer, split_='=')
singer_grammar

{'singer': [['姓名', '程度副词*', '擅长', '特点']],
 '姓名': [['华晨宇'], ['毛不易']],
 '程度副词*': [['null'], ['程度副词', '程度副词*']],
 '程度副词': [['很'], ['非常'], ['特别'], ['极其']],
 '擅长': [['喜欢'], ['善于'], ['擅长']],
 '特点': [['作词'], ['作曲'], ['喝酒'], ['怼人']]}

In [49]:
generate(singer_grammar, 'singer')

'华晨宇极其特别喜欢作曲'

In [50]:
generate_n(singer_grammar, 'singer', 6)

['华晨宇特别喜欢作曲',
 '毛不易特别很很非常很很极其极其善于作词',
 '毛不易很善于喝酒',
 '毛不易善于喝酒',
 '华晨宇特别喜欢怼人',
 '华晨宇特别善于怼人']

In [51]:
self_introduction = """
self_introduction = 礼貌称呼 报姓名 来处 主修 喜欢 具体爱好*
礼貌称呼 = null | 您好， | 你们好， | 老师们好，
报姓名 = 我 动词 姓名
动词 = 是 | 叫
姓名 = xhp， | bxf，
来处 = 来自 学校
学校 = 同济大学， | 华东师范大学， | 中南大学，
主修 = null | 专业是 专业
专业 = 植物学， | 采矿工程， | 土木工程，
喜欢 = 喜欢 | 擅长 | 善于
具体爱好* = 具体爱好 | 具体爱好*
具体爱好 = 听歌 | 码代码 | 弹钢琴 | 画画 | 跑步 
"""

In [52]:
self_introduction_grammar = create_grammar(self_introduction, split_='=')
self_introduction_grammar

{'self_introduction': [['礼貌称呼', '报姓名', '来处', '主修', '喜欢', '具体爱好*']],
 '礼貌称呼': [['null'], ['您好，'], ['你们好，'], ['老师们好，']],
 '报姓名': [['我', '动词', '姓名']],
 '动词': [['是'], ['叫']],
 '姓名': [['xhp，'], ['bxf，']],
 '来处': [['来自', '学校']],
 '学校': [['同济大学，'], ['华东师范大学，'], ['中南大学，']],
 '主修': [['null'], ['专业是', '专业']],
 '专业': [['植物学，'], ['采矿工程，'], ['土木工程，']],
 '喜欢': [['喜欢'], ['擅长'], ['善于']],
 '具体爱好*': [['具体爱好'], ['具体爱好*']],
 '具体爱好': [['听歌'], ['码代码'], ['弹钢琴'], ['画画'], ['跑步']]}

In [53]:
generate(self_introduction_grammar, 'self_introduction')

'老师们好，我是bxf，来自华东师范大学，专业是土木工程，擅长弹钢琴'

In [54]:
generate_n(self_introduction_grammar, 'self_introduction', 5)

['你们好，我叫xhp，来自同济大学，专业是土木工程，善于跑步',
 '你们好，我叫bxf，来自华东师范大学，善于弹钢琴',
 '老师们好，我叫xhp，来自同济大学，善于画画',
 '老师们好，我叫bxf，来自华东师范大学，善于弹钢琴',
 '你们好，我叫xhp，来自华东师范大学，专业是植物学，擅长跑步']

##### 2.2.2 使用新数据源完成语言模型的训练

In [55]:
movie_comments = pd.read_csv('../../datasource-master/movie_comments.csv')

C:\Users\xxx\Anaconda3\envs\daily\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
movie_comments.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [57]:
comments = movie_comments['comment'].tolist()
comments[:5]

['吴京意淫到了脑残的地步，看了恶心想吐',
 '首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮番上场，视物理逻辑于不顾，不得不说有钱真好，随意胡闹',
 '吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋律，为了煽情而煽情，让人觉得他是个大做作、大谎言家。（7.29更新）片子整体不如湄公河行动，1.整体不够流畅，编剧有毒，台词尴尬；2.刻意做作的主旋律煽情显得如此不合时宜而又多余。',
 '凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。',
 '中二得很']

In [58]:
len(comments)

261497

In [59]:
import re

In [60]:
def token(comments):
    res = []
    for c in comments:
        try:
            res.append(''.join(re.findall('\w+', c)))
        except TypeError:
            print(c)
    return ''.join(res)

In [61]:
tokens = token(comments)

nan
nan


In [62]:
tokens[:100]

'吴京意淫到了脑残的地步看了恶心想吐首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹吴京的炒作水平不输冯小刚但小刚至少不'

In [63]:
comments_cut = list(jieba.cut(tokens))

In [64]:
comments_count = Counter(comments_cut)

In [65]:
comments_count.most_common(10)

[('的', 328253),
 ('了', 102408),
 ('是', 73433),
 ('我', 50520),
 ('都', 36251),
 ('很', 34760),
 ('看', 33850),
 ('电影', 33638),
 ('也', 32064),
 ('和', 31291)]

In [66]:
comments_2_gram = [comments_cut[i]+comments_cut[i+1] for i in range(len(comments_cut)-1)]
comments_2_gram[:10]

['吴京意淫', '意淫到', '到了', '了脑残', '脑残的', '的地步', '地步看', '看了', '了恶心', '恶心想']

In [67]:
comments_2_gram_count = Counter(comments_2_gram)

In [68]:
comments_2_gram_count.most_common(10)

[('的电影', 8631),
 ('看的', 7075),
 ('都是', 6335),
 ('让人', 5278),
 ('的故事', 4707),
 ('看了', 4538),
 ('也是', 4407),
 ('的时候', 4398),
 ('的是', 4348),
 ('的人', 4344)]

In [69]:
prob_2('好', '电影', comments_2_gram_count)

8.959654515721874e-05

In [70]:
comments_compare = [
    '这个电影真好看 这个电影正好看',
    '晚上一起去看电影啊 一起去看晚上的电影啊'
]
for comment in comments_compare:
    c1, c2 = comment.split()
    p1, p2 = map(partial(get_probability, gram1=comments_count, gram2=comments_2_gram_count), [c1, c2])
    better = c1 if p1 > p2 else c2
    print('{} is more possible'.format(better))
    print('-'*4 + '{} with probability {}'.format(c1, p1))
    print('-'*4 + '{} with probability {}'.format(c2, p2))

这个电影真好看 is more possible
----这个电影真好看 with probability 1.409162345271777e-09
----这个电影正好看 with probability 2.099507362542209e-11
晚上一起去看电影啊 is more possible
----晚上一起去看电影啊 with probability 3.7070726842844085e-14
----一起去看晚上的电影啊 with probability 3.413842074613796e-16


In [71]:
def generate_best(grammar, target, n):
    sentences = generate_n(grammar, target, n)
    sentences_with_prob = sorted(zip(sentences, map(partial(get_probability, gram1=comments_count, gram2=comments_2_gram_count), sentences)), key=lambda x: x[1])
    print(sentences_with_prob)
    return sentences_with_prob[-1][0]

In [72]:
generate_best(self_introduction_grammar, 'self_introduction', 5)

[('老师们好，我叫xhp，来自中南大学，专业是植物学，擅长弹钢琴', 3.1392659662122783e-34), ('您好，我叫bxf，来自中南大学，专业是土木工程，擅长码代码', 5.44767466909249e-29), ('你们好，我叫xhp，来自华东师范大学，专业是植物学，善于跑步', 4.221554451666974e-27), ('老师们好，我叫bxf，来自中南大学，善于弹钢琴', 1.2248645572774896e-25), ('您好，我叫xhp，来自中南大学，善于画画', 6.565710838995328e-19)]


'您好，我叫xhp，来自中南大学，善于画画'

* Q: 这个模型有什么问题？ 你准备如何提升？

* Ans: <br>
1.基于规则，只适用于特定的场景特定的角色，不够通用；生成的句子中所选用的词全部来源于语法定义中的列表，词源比较有限，生成的句子较为单一。如果仍然是在基于规则的基础上提升的话，只能定义更多更完整的语法，定义更加完整的待选词列表 <br>
2.语言模型不够全面，只是针对电影评论生成的，所以求得的句子合理的概率会有偏差；增加语料库的丰富性、多样性 <br>
3.完善语料库、增加语法的丰富性

### 3 (Optional) 完成基于Pattern Match的语句问答

#### 3.1 Pattern Match

In [98]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [99]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return '', ''
        else: return pat_match(pattern[1:], saying[1:])

In [100]:
pat_match('?'.split(), 'jkljas hkjaks'.split())

('?', 'jkljas')

In [101]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    if is_variable(pattern[0]): 
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    elif pattern[0] != saying[0]:
        return []
    else:
        return pat_match(pattern[1:], saying[1:])

In [102]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [103]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [104]:
def substitute(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])]+substitute(rule[1:], parsed_rules)

In [105]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
got_patterns

[('?X', 'iPhone')]

In [106]:
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [107]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())
john_pat

[('?P', 'John'), ('?X', 'resting')]

In [108]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [109]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [110]:
substitute('why does ?P need ?X ?'.split(), pat_to_dict(john_pat))

['why', 'does', 'John', 'need', 'vacation', '?']

In [111]:
' '.join(substitute('why does ?P need ?X ?'.split(), pat_to_dict(john_pat)))

'why does John need vacation ?'

In [112]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [113]:
def get_response(saying, rules):
    for q, a in rules.items():
        parsed_rules = pat_match(q.split(), saying.split())
        if parsed_rules:
            resp = np.random.choice(a)
            return ' '.join(substitute(resp.split(), pat_to_dict(parsed_rules)))
    return ''

In [114]:
get_response('I need iPhone', defined_patterns) 

'Why do you need iPhone ?'

In [115]:
get_response("My mother told me something", defined_patterns)

'How do you think about your mother ?'

#### Segment Match

In [116]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [117]:
is_pattern_segment('?*P')

True

In [118]:
from collections import defaultdict

In [119]:
def is_match(rest, saying):
    if not rest or not saying:
        return True
#     elif not rest: return False
#     elif not saying: return False
    elif not all(s.isalpha() for s in rest[0]): # 如果当前 pattern 第一个是 variable 或者 pattern_segment，则后面的匹配可交给下一次继续进行
        return True
    elif rest[0] != saying[0]:
        return False
    else:
        return is_match(rest[1:], saying[1:])

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    
    if not rest: return (seg_pat, saying), len(saying)
    
    for i, token in enumerate(saying):
        # 扩增到直至后面的可匹配
        if rest[0] == token and is_match(rest[1:], saying[i+1:]):
#             print(token, seg_pat)
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

In [120]:
segment_match('?*PL are good'.split(), 'My dog are good too'.split())

(('?PL', ['My', 'dog']), 2)

In [121]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    # 修改了此处，阻止"?*X hello ?*Y"和"Hi, how do you do"的匹配，即匹配不允许saying或pattern有剩余，有剩余即为不能匹配
    if not pattern and not saying: return []
    elif not pattern: return fail
    elif not saying and all(s.isalpha() for s in pattern[0]): return fail
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [122]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [123]:
segment_match('hello world boring'.split(), "hello world".split())

(('hello', ['hello']), 1)

In [124]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [125]:
pat_match_with_seg('I need ?*X'.split(), 'I need an iphone'.split())

[('?X', ['an', 'iphone'])]

In [126]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [127]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [128]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [129]:
pat_match_with_seg("?*X hello ?*Y".split(), "Hi, how do you do".split())

[('?X', ['Hi,', 'how', 'do', 'you', 'do']), True, None]

In [130]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

* 问题1

In [131]:
def get_response(saying, response_rules):
    for q, a in response_rules.items():
        match = pat_match_with_seg(q.split(), saying.split())
        if match[-1] != None:
            print(match)
            resp = np.random.choice(a)
            return ' '.join(substitute(resp.split(), pat_to_dict(match)))
    return ''

In [132]:
get_response('I was xhp', rules)

[('?X', ['xhp'])]


'Were you really xhp ?'

In [133]:
get_response('hello', rules)

[('?X', []), ('?Y', [])]


'Hi, how do you do?'

* 问题2

In [134]:
def split_chinese(sentence):
    res = []
    is_pattern = False
    j = 0
    for i in range(len(sentence)):
        if not is_pattern and ord(sentence[i]) < 128:
            tmp = sentence[j:i]
            if tmp: res.append(tmp)
            j = i
            is_pattern = True
        if is_pattern and ord(sentence[i]) >= 128:
            tmp = sentence[j:i]
            if tmp: res.append(tmp)
            j = i
            is_pattern = False
    res.append(sentence[j:])
    return res

In [135]:
split_chinese('?*x我想要?*y')

['?*x', '我想要', '?*y']

In [136]:
def cut_chinese(sentence):
    tokens = list(jieba.cut(sentence))
    length = len(tokens)
    i, j, res = 0, 0, []
    while i < length:
        if len(tokens[i]) > 1 or ord(tokens[i]) >= 128:
            if i > j: 
                res.append(''.join(tokens[j:i]))
                j = i
            res.append(tokens[i])
            j += 1
        i += 1
    if i > j: res.append(''.join(tokens[j:i]))
    return res  

In [137]:
cut_chinese('?*x我想要?*y')

['?*x', '我', '想要', '?*y']

In [138]:
def pat_to_dict(patterns):
    return {k: ''.join(v) if isinstance(v, list) else v for k, v in patterns}

In [139]:
def get_ch_response(saying, response_rules):
    for q, a in response_rules.items():
        match = pat_match_with_seg(cut_chinese(q), cut_chinese(saying))
        print(match)
        if match[-1] != None:
            resp = np.random.choice(a)
            return ''.join(substitute(cut_chinese(resp), pat_to_dict(match)))
    return ''

In [140]:
chinese_rules = {
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想和?y在一起吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？']
}

In [141]:
get_ch_response('我喜欢毕行风', chinese_rules)

[('?x', ['我', '喜欢', '毕', '行风']), True, None]
[('?x', ['我']), ('?y', ['毕', '行风'])]


'毕行风有什么好的呢？'

In [142]:
get_ch_response('我想要一杯奶茶', chinese_rules)

[('?x', []), ('?y', ['一杯', '奶茶'])]


'想问你，你觉得一杯奶茶有什么意义呢?'

* 问题3

通过修改程序实现一个可以多轮对话的机器人

In [143]:
def mul_rounds_dialogue(response_rules):
    while True:
        saying = input('you: ')
        if saying in ['q', 'exit']: break
        for q, a in response_rules.items():
            match = pat_match_with_seg(cut_chinese(q), cut_chinese(saying))
            if match[-1] != None:
                resp = np.random.choice(a)
                resp = ''.join(substitute(cut_chinese(resp), pat_to_dict(match)))
                print('bot: {}'.format(resp))
                break
        else:
            print('bot: 我词穷了 :-(')
    return 

In [146]:
dialogue_rules = {
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想要?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*y很好玩啊': ['哪里好玩了'],
    '?*y很好啊': ['好在哪啊'],
    '?*y很有用啊': ['可以用在哪啊，比如...'],
    '?*x一点都不像': ['你眼瞎啊', '好吧'],
    '?*x哪都好啊': ['那你说说呗😏', '列举一下啊'],
    '?*优点多着呢': ['那你说说呗', '列举一下啊'],
    '?*x想啊?*y': ['祝你好运，哈哈哈'],
    '?*x当然?*y': ['祝你好运，哈哈哈，😄'],
    '?*x哪都喜欢': ['服了你了。。。'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想和?y在一起吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？']
}

In [147]:
mul_rounds_dialogue(dialogue_rules)

you: 我喜欢毕行风
bot: 毕行风有什么好的呢？
you: 哪都好啊
bot: 那你说说呗😏
you: 好看啊、性格又好、又努利
bot: 我词穷了 :-(
you: 哈哈哈哈
bot: 我词穷了 :-(
you: q


* 问题4

1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？<br>
Ans: <br>
   * 优点：不需要经过长时间的训练就可以得到一个简易的对话模型，不需要深厚的专业知识就可以写出各种各样的基于匹配的对话程序
   * 缺点： 无法进行持续对话，回答模式过于单一需要人工事先定义

2.什么是数据驱动？数据驱动在这个程序里如何体现？<br>
Ans: <br>
   * 我理解的数据驱动就是不挖掘出数据中潜藏的规律和模式，以此来改善我们通过程序获得的结果
   * 在这个程序中如果我们定义足够多的样板模式，并对每一种样本模式尽可能详尽的列举可能的回答，那么我们也可以获得非常好的结果

3.数据驱动与 AI 的关系是什么？<br>
Ans: 数据可以说是 AI 的基本资料，以上实现的程序也需要我们事先定义好的规则、模式这些数据，然后根据模式通过程序实现匹配实现一个简易的对话机器人，；而现在的大部分 AI 程序则实现了自主探索数据中潜藏的规律，并不需要人工发现，这就对数据量提出了更大的要求，足够多的数据才具有统计意义